# Model Setup Script with Clouds 

You should already understand the basics of a `model_setup.py` file. Here we will show how to add a case with a cloud retrieval. 

## Declare Directories

Here, we are grabbing the planet parameters from a google sheet which will make our lives much easier as it will automatically pull the parameters needed.

A google sheet is not the only way of storing and pulling parameters, but it is the method used her



In [1]:
import numpy as np
import os 
import picaso.justdoit as jdi
import picaso.analyze as lyz


sheet_id = '1R3DlcC25MyfP97DNcbfsy1dNehu20BWal8ePOol9Ftg'
sheet_name = '0'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={sheet_name}"


## Set Up Opacity

Define the opacities needed to run the climate grid with.

Here we set up opacity as a dictionary incase you want to run multiple opacities.

In [ ]:
opacity = {
            'db1':jdi.opannection( filename_db="/data2/picaso_dbs/R15000/all_opacities_0.3_15_R15000.db", wave_range=[0.3,15])
}

## Observational Data and Parameters

We also need to point to the observational data.

Let's point towards the grid locations, create a grid fitter object for them, and prep them. 

The basic premise of `prep_gridtrieval`:
- vets and transforms the grid to ensure it's square and interprelate-able
- checks that there is a common pressure grid for the temperature

In [ ]:
location = "/data2/models/HAT-P-26b/spec/"
grid_name = 'cldfree'
fitter = lyz.GridFitter(grid_name,location, verbose=True, to_fit='fpfs_emission', save_chem=True)
fitter.prep_gridtrieval(grid_name)

data_dir = '/data2/observations/HAT-P-26b/'

Finally, let's define our grid parameters and create them for the fitter. Also, let's pull our planet/star parameters from the google sheet.

In [ ]:
grid_parameters_unique = fitter.interp_params[grid_name]['grid_parameters_unique']
grid_points = fitter.interp_params[grid_name]['grid_parameters']


success = jdi.pd.read_csv(url)
log_g = success.loc[0,'logg']
metallicity =success.loc[0,'feh']
t_eff = success.loc[0,'st_teff'] #K
r_star = success.loc[0,'rstar'] #rsun
m_planet = success.loc[0,'pl_mass'] #mjup
r_planet =success.loc[0,'pl_rad']  #rjup

## Setting up planet model

Next, let's set up our planet model. This is creating the planet model for each opacity we run through and saving it into an array of picaso classes.

In [ ]:
def setup_planet():
    """
    First need to setup initial parameters. Usually these are fixed (anything we wont be including
    in the retrieval would go here).
    
    Returns
    -------
    Must return the full picaso class
    """
    pl = {i:jdi.inputs() for i in opacity.keys()}
    for i in opacity.keys(): pl[i].star(opacity[i], t_eff,metallicity,log_g,
                                        radius=r_star,
                                        database = 'phoenix',
                                        radius_unit = jdi.u.Unit('R_sun') )
    #i am putting these here but in a lot of cases, this should go into the likelihood function
    #for example when adjusting radius through xrp factor
    #or if fitting for planet mass, radius, or gravity
    for i in opacity.keys(): pl[i].approx(p_reference=1)
    for i in opacity.keys(): pl[i].gravity(mass=m_planet, mass_unit=jdi.u.Unit('M_jup'),
              radius=r_planet, radius_unit=jdi.u.Unit('R_jup'))
    for i in opacity.keys(): pl[i].phase_angle(0)
    return pl

planet_og = setup_planet()

## Setting up Ultranest

This is simply keeping track of what parameters are used in each run.

## Param Set

In [1]:
class param_set:
    """
    This is purely for book keeping what parameters you have run in each retrieval.
    It helps if you keep variables uniform.
    THINGS TO CHECK:
    1) Make sure that the variables here match how you are unpacking your cube in the model_set class and prior_set
    2) Make sure that the variable names here match the function names in model_set and prior_set
    """
    #line='m,b' #simplest test model = blackbody: blackbody='Tiff'
    cld_free = ','.join(grid_parameters_unique.keys())
    #now lets add a new model to explore. We want to fit for other parameters besides just the grid parameters 
    w_clds = ','.join(grid_parameters_unique.keys())+',fsed,kzz' #

print(param_set.w_clds)

NameError: name 'grid_parameters_unique' is not defined

## Guesses Set

In testing, it is very useful to check that it is grabbing the right parameters before doing a full analysis. This is available  for a sanity check if desired

In [ ]:
class guesses_set: 
    """
    Optional! 
    This is only used if you want to test your initial functions before running 
    the full retrievals. See script test.py
    """
    #completely random guesses just to make sure it runs
    cld_free=[grid_parameters_unique[i][0]
             for i in grid_parameters_unique.keys()]# + [-.005,-.005]

## Model Set

Here, we are defining the full model. This is essentially prepping and making it easy to digest for Ultranest's `cube` usage.

In [ ]:
class model_set:
    """100e-6=1e-4
    This is your full model set. It will include all the functions you want to test
    for a particular data set.
    """     
    def cld_free(cube): 
        final_goal = cube[0:len(grid_parameters_unique.keys())]
        spectra_interp = lyz.custom_interp(final_goal, fitter, grid_name)
        micron = fitter.wavelength[grid_name]
        wno = 1e4/fitter.wavelength[grid_name] 
        return wno, spectra_interp

## Prior Set

Finally, we are storing all the priors for Ultranest to use.

In [ ]:
class prior_set:
    """
    Store all your priors. You should have the same exact function names in here as
    you do in model_set and param_set

    Make sure the order of the unpacked cube follows the unpacking in your model 
    set and in your parameter set. 
    #pymultinest: http://johannesbuchner.github.io/pymultinest-tutorial/example1.html
    """   
    def cld_free(cube):#,ndim, nparams):
        params = cube.copy()
        for i,key in enumerate(grid_parameters_unique): 
            minn = np.min(grid_parameters_unique[key]) 
            maxx = np.max(grid_parameters_unique[key]) 
            params[i] = minn + (maxx-minn)*params[i]
        return params